# Baseline

<b> Построение бейзлайн модели является довольно важным моментом, так как именно ориентируясь на бейзлайн и будет происходить дальнейшие изменения.   
В нашем случае будет предсказываться стоимость жилья в зависимости от его параметров (площадь, количество комнат, геопозиция, регион и тд)</b>

In [68]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [111]:
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [70]:
from lightgbm import LGBMRegressor

In [71]:
data = pd.read_csv("all_v2.csv")

<b> С помощью EDA было выяснено, что в датасете содержатся некорректны данные, а также большое количество выбросов.  </b>

<b> Проведем чистку данных. Чтобы убрать ошибочные значения, которые в дальнейшем могут ухудшить качество модели. </b>

In [72]:
data = data[(data.price > 0)&(data.rooms!=-2)&(data.area>0)&(data.kitchen_area>0)]
data = data.drop_duplicates()
data.reset_index(inplace=True,drop=True)

<b> Уберем регионы, у которых очень мало значений. </b>

In [73]:
regions_to_drop = data.groupby('region')['region'].count().sort_values().head(40).index.to_list()
rows_to_drop = np.array([])
for region in regions_to_drop:
    rows_to_drop = np.append(rows_to_drop,np.where(data.region==region)[0])
data.drop(rows_to_drop,axis=0,inplace=True)
data.reset_index(inplace=True,drop=True)

<b> Функция для чистки данных от выбросов. Функия убирает из датасета строки с экстремальными значениями  (более или менее 3х среднеквадратичных отклонений) для каждого региона. </b>

In [74]:
def norm_by_region(data,feature,region):
        std_shift_3 = data.loc[data.region==region][feature].std() * 3
        feat_mean = data.loc[data.region==region][feature].mean()
        lower_bound = feat_mean-std_shift_3
        upper_bound = feat_mean+std_shift_3
        rows_to_drop = np.where((data.loc[data.region==region,feature]<lower_bound) | \
                                (data.loc[data.region==region,feature]>=upper_bound))[0]
        indexes = data.loc[data.region==region].iloc[rows_to_drop].index
        data.drop(indexes,axis=0,inplace=True)
        data.reset_index(inplace=True,drop=True)
        return data

In [75]:
for region in tqdm(data.region.unique()):
    data = norm_by_region(data,"price",region)
    data = norm_by_region(data,"area",region)

100%|██████████| 44/44 [01:00<00:00,  1.38s/it]


<b> Преобразуем даты в целое число дней с момента начала данных, для того чтобы привести признаковые описания к одному виду. </b>

In [76]:
data["date_time"] = data["date"] + " " + data["time"]
data = data.drop(["date","time"],axis=1)

In [77]:
data['date_time'] = pd.to_datetime(data["date_time"])
data["day_delta"] = (data["date_time"] - data.date_time.min()).dt.days
data = data.drop("date_time",axis=1)

<b> Преобразуем бинарный признак. </b>

In [78]:
data["object_type"] = data["object_type"].map({1:0,11:1}).astype(int)

<b> One hot encoding для категориальных признаков. </b>

In [79]:
building_types = pd.get_dummies(data["building_type"],prefix="buiilding_type_")
regions = pd.get_dummies(data["region"],prefix="region_")
data = data.drop("building_type",axis=1)
data = data.drop("region",axis=1)

<b> Делим выборку на обучающую и тестовую с учетом временного ряда, в соотношении 70/30. </b>

In [80]:
data = data.sort_values(by="day_delta")
train = data.iloc[:round(data.shape[0]*0.7)]
test = data.iloc[round(data.shape[0]*0.7):]
y_train = train.price
y_test = test.price

In [81]:
train = train.drop("price",axis=1)
test = test.drop("price",axis=1)

<b> Проведем нормализацию данных для того,чтобы исключить дисбаланс между значениями признаков и привести их к одному порядку. Это положительно отразится на устойчивости работы модели. </b>

In [82]:
scaler = RobustScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

<b> Присоединяем категориальные фичи после скалирования выборок. </b>

In [83]:
train_scaled = np.hstack((train_scaled,building_types.values[:round(data.shape[0]*0.7)]))
train_scaled = np.hstack((train_scaled,regions.values[:round(data.shape[0]*0.7)]))
test_scaled = np.hstack((test_scaled,building_types.values[round(data.shape[0]*0.7):]))
test_scaled = np.hstack((test_scaled,regions.values[round(data.shape[0]*0.7):]))

<b> Построим baseline модель, чтобы в дальнейшем анализировать прирост качества в зависимости с усложнением моделей. В качестве метрики используем среднее абсолютное отклонение (mean absolute error (MAE)). Метрика довольно легко рассчитывается, и она наглядна. </b>

In [113]:
%%time
linear = LinearRegression()
linear.fit(train_scaled,y_train)

CPU times: user 41.9 s, sys: 1.8 s, total: 43.7 s
Wall time: 15.8 s


LinearRegression()

In [114]:
round(mean_absolute_error(y_test,linear.predict(test_scaled)))

2044101.0

In [115]:
round(mean_absolute_error(y_train,linear.predict(train_scaled)))

1549607.0

<b> Проверим как изменится качество при использовании более сложной модели.  
Более сложная модель машинного обучения показывает лучше результат, однако время обучения и прогноза возрастает. </b>

In [110]:
%%time
lgb = LGBMRegressor()
lgb.fit(train_scaled, y_train)

CPU times: user 45.9 s, sys: 563 ms, total: 46.5 s
Wall time: 6.73 s


LGBMRegressor()

In [101]:
round(mean_absolute_error(y_test, lgb.predict(test_scaled)))

1040824.0

In [102]:
round(mean_absolute_error(y_train, lgb.predict(train_scaled)))

648948.0

<b> Линейная регрессия может выступать в качестве бейзлайна. Модель относительно простая и её легко объяснить. Сравнивая её с градиентным бустингом точность модели ниже, но она превосходит в скорости обучения и прогнозирования. Признаковые преобразования и чистка данных от выбросов может значительно увеличить её качество (а также подбор гиперпараметров).  </b>